<a href="https://colab.research.google.com/github/ParalelaUCM/biciMAD/blob/master/triciclos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!pip install pyspark

In [0]:
import json
from pyspark import SparkContext
sc = SparkContext()

In [275]:
rdd2 = sc.textFile("/content/g2.txt").\
    map(lambda x: x.replace('"',"").replace(' ',"").split(",")).\
    map(lambda x: ("berni",x)).groupByKey().map(lambda x: (x[0],list(x[1])))
rdd2.collect()

[('berni',
  [['B', 'B'],
   ['B', 'A'],
   ['C', 'A'],
   ['A', 'B'],
   ['A', 'D'],
   ['B', 'C'],
   ['F', 'A'],
   ['F', 'D']])]

In [276]:
rdd3 = sc.textFile("/content/g3.txt").\
    map(lambda x: x.replace('"',"").replace(' ',"").split(",")).\
    map(lambda x: ("sergi",x)).groupByKey().map(lambda x: (x[0],list(x[1])))
rdd3.collect()

[('sergi',
  [['A', 'B'], ['A', 'C'], ['A', 'D'], ['A', 'F'], ['F', 'F'], ['F', 'A']])]

In [277]:
rdd4 = sc.textFile("/content/g4.txt").\
    map(lambda x: x.replace('"',"").replace(' ',"").split(",")).\
    map(lambda x: ("eli",x)).groupByKey().map(lambda x: (x[0],list(x[1])))
rdd4.collect()

[('eli',
  [['A', 'B'],
   ['B', 'C'],
   ['C', 'D'],
   ['D', 'E'],
   ['E', 'F'],
   ['F', 'A'],
   ['A', 'C'],
   ['C', 'E'],
   ['E', 'A']])]

In [278]:
rdd1 = sc.textFile("/content/g1.txt").\
    map(lambda x: x.replace('"',"").replace(' ',"").split(",")).\
    map(lambda x: ("pablo",x)).groupByKey().map(lambda x: (x[0],list(x[1])))
rdd1.collect()

[('pablo',
  [['A', 'B'],
   ['A', 'B'],
   ['A', 'C'],
   ['A', 'D'],
   ['B', 'C'],
   ['B', 'D'],
   ['C', 'D']])]

In [279]:
rdd10 = sc.textFile("/content/g10.txt").\
    map(lambda x: x.replace('"',"").replace(' ',"").split(",")).\
    map(lambda x: ("marcos",x)).groupByKey().map(lambda x: (x[0],list(x[1])))
rdd10.collect()

[('marcos',
  [['1', '2'],
   ['2', '3'],
   ['3', '4'],
   ['1', '3'],
   ['4', '5'],
   ['5', '6'],
   ['3', '5'],
   ['6', '7'],
   ['7', '8'],
   ['5', '7'],
   ['8', '9']])]

In [0]:
def adic(ldl):
  d = {}
  for elem in ldl:
    key,value = elem[0],elem[1]
    if key in d:
      d[key].append(value)
    else:
      d[key]=[value]
  return d

In [0]:
def unomas(dic):
  dk = list(dic.keys())
  for key in dk:
    for elem in dic[key]:
      if key[0]==elem:
        dic[key+elem]=[len(key+elem)-1]
      elif str(elem) in key:
        pass
      else:
        try:
          dic[key+elem]=dic[elem]
        except KeyError:
          pass
  return dic

In [0]:
def sigue(dic):
  if len(dic) == len(unomas(dic)):
    return dic
  else:
    return (sigue(unomas(dic)))

In [0]:
def dameciclos(dic,n):
  resultado = []
  for key,value in dic.items():
    if value == [n]:
      resultado.append(key)
  return resultado

In [0]:
rdd = rdd2.union(rdd3).union(rdd4).union(rdd1)#.union(rdd10)
rdd2 = rdd.mapValues(lambda x: adic(x)).mapValues(lambda x: sigue(x))

In [285]:
rdd2.mapValues(lambda x: dameciclos(x,8)).collect()

[('berni', []), ('sergi', []), ('eli', []), ('pablo', [])]

In [286]:
#todos los ciclos menores que m:
m=6
for i in range(1,m):
  a = rdd2.mapValues(lambda x: dameciclos(x,i))
  print("ciclos de longitud "+str(i)+": ",a.collect())

ciclos de longitud 1:  [('berni', ['BB']), ('sergi', ['FF']), ('eli', []), ('pablo', [])]
ciclos de longitud 2:  [('berni', ['BAB', 'ABA']), ('sergi', ['AFA', 'FAF']), ('eli', []), ('pablo', [])]
ciclos de longitud 3:  [('berni', ['BCAB', 'CABC', 'ABCA']), ('sergi', []), ('eli', ['ACEA', 'CEAC', 'EACE']), ('pablo', [])]
ciclos de longitud 4:  [('berni', []), ('sergi', []), ('eli', ['ABCEA', 'ACDEA', 'ACEFA', 'BCEAB', 'CDEAC', 'CEFAC', 'CEABC', 'DEACD', 'EFACE', 'EABCE', 'EACDE', 'FACEF']), ('pablo', [])]
ciclos de longitud 5:  [('berni', []), ('sergi', []), ('eli', ['ABCDEA', 'ABCEFA', 'ACDEFA', 'BCDEAB', 'BCEFAB', 'CDEFAC', 'CDEABC', 'CEFABC', 'DEFACD', 'DEABCD', 'EFABCE', 'EFACDE', 'EABCDE', 'FABCEF', 'FACDEF']), ('pablo', [])]


In [287]:
dos = unomas({'A': ['B', 'D'], 'B': ['B', 'A', 'C'], 'C': ['A'], 'F': ['A', 'D']})
dos

{'A': ['B', 'D'],
 'AB': ['B', 'A', 'C'],
 'B': ['B', 'A', 'C'],
 'BA': ['B', 'D'],
 'BB': [1],
 'BC': ['A'],
 'C': ['A'],
 'CA': ['B', 'D'],
 'F': ['A', 'D'],
 'FA': ['B', 'D']}

In [288]:
tres = unomas(dos)
tres

{'A': ['B', 'D'],
 'AB': ['B', 'A', 'C'],
 'ABA': [2],
 'ABC': ['A'],
 'B': ['B', 'A', 'C'],
 'BA': ['B', 'D'],
 'BAB': [2],
 'BB': [1],
 'BC': ['A'],
 'BCA': ['B', 'D'],
 'C': ['A'],
 'CA': ['B', 'D'],
 'CAB': ['B', 'A', 'C'],
 'F': ['A', 'D'],
 'FA': ['B', 'D'],
 'FAB': ['B', 'A', 'C']}

In [289]:
cuatro = unomas(tres)
cuatro

{'A': ['B', 'D'],
 'AB': ['B', 'A', 'C'],
 'ABA': [2],
 'ABC': ['A'],
 'ABCA': [3],
 'B': ['B', 'A', 'C'],
 'BA': ['B', 'D'],
 'BAB': [2],
 'BB': [1],
 'BC': ['A'],
 'BCA': ['B', 'D'],
 'BCAB': [3],
 'C': ['A'],
 'CA': ['B', 'D'],
 'CAB': ['B', 'A', 'C'],
 'CABC': [3],
 'F': ['A', 'D'],
 'FA': ['B', 'D'],
 'FAB': ['B', 'A', 'C'],
 'FABC': ['A']}

In [290]:
cinco = unomas(cuatro)
cinco

{'A': ['B', 'D'],
 'AB': ['B', 'A', 'C'],
 'ABA': [2],
 'ABC': ['A'],
 'ABCA': [3],
 'B': ['B', 'A', 'C'],
 'BA': ['B', 'D'],
 'BAB': [2],
 'BB': [1],
 'BC': ['A'],
 'BCA': ['B', 'D'],
 'BCAB': [3],
 'C': ['A'],
 'CA': ['B', 'D'],
 'CAB': ['B', 'A', 'C'],
 'CABC': [3],
 'F': ['A', 'D'],
 'FA': ['B', 'D'],
 'FAB': ['B', 'A', 'C'],
 'FABC': ['A']}

In [291]:
cuatro==cinco

True